In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shashwatwork/knee-osteoarthritis-dataset-with-severity")

print("Path to dataset files:", path)

Path to dataset files: /home/jak5je/.cache/kagglehub/datasets/shashwatwork/knee-osteoarthritis-dataset-with-severity/versions/1


In [2]:
import torch
print(torch.version.cuda)          # CUDA version PyTorch was built with
print(torch.cuda.get_device_name(0))  # Your GPU model
print(torch.cuda.is_available())   # Should be True if CUDA works


12.4
NVIDIA RTX A6000
True


In [4]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image

In [5]:
path = "/home/jak5je/.cache/kagglehub/datasets/shashwatwork/knee-osteoarthritis-dataset-with-severity/versions/1"
train_dir = os.path.join(path, "train")
val_dir = os.path.join(path, "val")
test_dir = os.path.join(path, "test")

In [6]:
class KneeOADataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        
        for label in range(5):  # Labels: 0 to 4
            label_dir = os.path.join(root_dir, str(label))
            if os.path.exists(label_dir):
                for img_name in os.listdir(label_dir):
                    self.image_paths.append(os.path.join(label_dir, img_name))
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [14]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [15]:
train_dataset = KneeOADataset(train_dir, transform=train_transform)
val_dataset = KneeOADataset(val_dir, transform=transform)
test_dataset = KneeOADataset(test_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.densenet121(pretrained=True)

/home/jak5je/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jak5je/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /home/jak5je/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 254MB/s]


In [11]:
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 5)
model = model.to(device)

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [18]:
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)
    return 100 * correct / total

In [19]:
epochs = 20
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
        
    train_acc = 100 * correct / total
    val_acc = evaluate(model, val_loader)
    scheduler.step()

    
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}, "
          f"Train Acc: {100 * correct/total:.2f}%, Val Acc: {val_acc:.2f}%")

# Save model
torch.save(model.state_dict(), "jack_densenet_knee_oa.pth")

Epoch 1, Loss: 0.7882, Train Acc: 67.06%, Val Acc: 62.35%
Epoch 2, Loss: 0.7777, Train Acc: 67.43%, Val Acc: 65.50%
Epoch 3, Loss: 0.7739, Train Acc: 68.02%, Val Acc: 60.41%
Epoch 4, Loss: 0.7610, Train Acc: 68.00%, Val Acc: 66.22%
Epoch 5, Loss: 0.7284, Train Acc: 68.90%, Val Acc: 60.41%
Epoch 6, Loss: 0.6645, Train Acc: 71.72%, Val Acc: 66.22%
Epoch 7, Loss: 0.6272, Train Acc: 73.97%, Val Acc: 65.74%
Epoch 8, Loss: 0.6087, Train Acc: 74.28%, Val Acc: 64.04%
Epoch 9, Loss: 0.5996, Train Acc: 74.80%, Val Acc: 63.92%
Epoch 10, Loss: 0.5875, Train Acc: 75.35%, Val Acc: 65.98%
Epoch 11, Loss: 0.5663, Train Acc: 76.51%, Val Acc: 65.13%
Epoch 12, Loss: 0.5553, Train Acc: 76.76%, Val Acc: 66.46%
Epoch 13, Loss: 0.5563, Train Acc: 77.35%, Val Acc: 65.25%
Epoch 14, Loss: 0.5526, Train Acc: 77.55%, Val Acc: 65.38%
Epoch 15, Loss: 0.5495, Train Acc: 76.98%, Val Acc: 65.50%
Epoch 16, Loss: 0.5374, Train Acc: 76.95%, Val Acc: 65.74%
Epoch 17, Loss: 0.5450, Train Acc: 77.15%, Val Acc: 65.86%
Epoch 